<a href="https://colab.research.google.com/github/nitsundon/100DaysofML/blob/main/Day01/LSTMrev02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import numpy as np


In [ ]:
df= pd.read_pickle("/content/drive/MyDrive/Libraries/pickle/preprocessed_demand_df.pkl")
df['datetime']=pd.to_datetime(df['datetime'])


In [ ]:
def create_datetime_features(df):

  df['year'] = df['datetime'].dt.year

  df['month'] = df['datetime'].dt.month
  df['day'] = df['datetime'].dt.day
  df['day_of_week'] = df['datetime'].dt.dayofweek
  df['day_name'] = df['datetime'].dt.day_name()
  df['quarter'] = df['datetime'].dt.quarter
  df['year'] = df['datetime'].dt.year
  df['block'] = 1+df['datetime'].dt.hour * 4 + df['datetime'].dt.minute // 15
  return df.copy();

In [ ]:
def add_cyclic_features(df, col, max_val,drop_original=True):
    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col]/max_val)
    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    if(drop_original):
      df.drop(col,axis=1,inplace=True)
    return df


In [ ]:
def create_lags(df,lag_steps):
  for i in range(1,lag_steps+1):
    df[f'demand_lag_{i}']=df['demand'].shift(i)
  return df

In [ ]:
def create_lstm_feature_and_target(df):
  features = df.drop('demand', axis=1)
  target = df['demand']
  return features,target

In [ ]:
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,0])
    return np.array(dataX),np.array(dataY)


